# IMU Data

## Acceleration

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
gyro_x = columns[3]
gyro_y = columns[4]
gyro_z = columns[5]
acc_x = columns[6]
acc_y = columns[7]
acc_z = columns[8]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

ax = plt.figure(figsize=(16,10))
plt.title(r"IMU Vertical Acceleration")
plt.ylabel(r"Measured Acceleration [m/s$^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(timestamp, acc_x)
ax.axes[0].set_xlim([-5,42])
#ax.axes[0].set_ylim([0.8,1.8])
plt.show()